In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Media_Bias_Feature_for_the_Identification_of_Biased_Statements_in_News_Articles')
!pwd

/content
/content/drive/MyDrive/Colab Notebooks/Media_Bias_Feature_for_the_Identification_of_Biased_Statements_in_News_Articles


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
df_nonpartisan = pd.read_csv("Datasets/news_headlines_usa_neutral.csv")

In [ ]:
df_nonpartisan

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url
0,663692044,2020-07-12 08:00:00,"""Don't Tase Me, Bro"" tops '07 memorable quote ...",http://www.reuters.com/article/us-quotes-odd-i...,en,False,NaN,4442,Reuters,http://www.reuters.com
1,668844035,2020-07-11 08:00:00,U.S. healthcare falls short in survey of 7 nat...,http://www.reuters.com/article/us-healthcare-s...,en,False,NaN,4442,Reuters,http://www.reuters.com
2,669761491,2020-07-10 08:00:00,Nicotine may ease Parkinson's symptoms: U.S. s...,https://www.reuters.com/article/us-parkinsons-...,en,False,NaN,4442,Reuters,http://www.reuters.com
3,680460617,2020-11-05 07:00:00,Experts Divided Over Safety of Indian Point Nu...,http://www.reuters.com/article/idUS38169798202...,en,False,NaN,4442,Reuters,http://www.reuters.com
4,803694205,2020-06-15 00:00:00,"Reports: Petraeus off the list, Trump down to ...",http://thehill.com/blogs/blog-briefing-room/ne...,en,False,NaN,18364,Hill,http://thehill.com/rss/syndicator/19109
...,...,...,...,...,...,...,...,...,...,...
83138,1868904207,2020-11-06 17:25:51,Biden's lead grows in Nevada as more results a...,https://apnews.com/article/election-2020-joe-b...,en,True,"politics and government, united states politic...",1154,AP,http://ap.org/
83139,1868915424,2020-04-07 13:27:56,Twitter's Dorsey sets up $1B fund; consumers g...,https://apnews.com/ffcd0103a51fc6fb3346176008b...,en,False,"finances, medicine and health, labor, insuranc...",1154,AP,http://ap.org/
83140,1868916918,2020-08-11 14:57:48,Fact check: The first Black U.S. president was...,https://www.reuters.com/article/uk-factcheck-f...,en,False,"politics and government, united states politic...",4442,Reuters,http://www.reuters.com
83141,1869115801,2020-06-29 20:03:59,Exclusive: Facebook ad boycott campaign to go ...,https://www.reuters.com/article/us-facebook-ad...,en,False,"advertising and marketing, advertising",4442,Reuters,http://www.reuters.com


In [ ]:
df_nonpartisan['media_name'].value_counts()

Reuters                                 22616
Yahoo News - Latest News & Headlines    16072
CNBC                                    12229
USA Today                               12086
AP                                      11052
Bloomberg                                8445
Hill                                      643
Name: media_name, dtype: int64

In [ ]:
df_nonpartisan_filtered = df_nonpartisan.loc[df_nonpartisan['media_name'].isin(['Reuters', 'CNBC', 'Hill'])]
df_nonpartisan_filtered['media_name'].value_counts()

Reuters    22616
CNBC       12229
Hill         643
Name: media_name, dtype: int64

In [ ]:
df_slight_partisan_filtered = df_nonpartisan.loc[df_nonpartisan['media_name'].isin(['Yahoo News - Latest News & Headlines', 'USA Today', 'AP'])]
df_slight_partisan_filtered['media_name'].value_counts()

Yahoo News - Latest News & Headlines    16072
USA Today                               12086
AP                                      11052
Name: media_name, dtype: int64

In [ ]:
df_nonpartisan_filtered['slant'] = 0
df_nonpartisan_filtered['nonpartisan'] = 1
df_nonpartisan_filtered['slight-partisan'] = 0
df_nonpartisan_filtered['hyperpartisan'] = 0
df_nonpartisan_filtered = df_nonpartisan_filtered.drop(['stories_id', 'publish_date', 'url', 'language', 'ap_syndicated', 'themes', 'media_id', 'media_name', 'media_url'], axis=1)
df_nonpartisan_filtered.head()

<ipython-input-8-2ac1f7d269f5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonpartisan_filtered['slant'] = 0
<ipython-input-8-2ac1f7d269f5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonpartisan_filtered['nonpartisan'] = 1
<ipython-input-8-2ac1f7d269f5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

,title,slant,nonpartisan,slight-partisan,hyperpartisan
0,"""Don't Tase Me, Bro"" tops '07 memorable quote ...",0,1,0,0
1,U.S. healthcare falls short in survey of 7 nat...,0,1,0,0
2,Nicotine may ease Parkinson's symptoms: U.S. s...,0,1,0,0
3,Experts Divided Over Safety of Indian Point Nu...,0,1,0,0
4,"Reports: Petraeus off the list, Trump down to ...",0,1,0,0


In [ ]:
df_nonpartisan_filtered.shape

(35488, 5)

In [ ]:
df_nonpartisan_filtered_rs = df_nonpartisan_filtered.sample(frac=0.20)

In [ ]:
df_nonpartisan_filtered_rs.shape

(7098, 5)

In [ ]:
df_slight_partisan_filtered['slant'] = 1
df_slight_partisan_filtered['nonpartisan'] = 0
df_slight_partisan_filtered['slight-partisan'] = 1
df_slight_partisan_filtered['hyperpartisan'] = 0
df_slight_partisan_filtered = df_slight_partisan_filtered.drop(['stories_id', 'publish_date', 'url', 'language', 'ap_syndicated', 'themes', 'media_id', 'media_name', 'media_url'], axis=1)
df_slight_partisan_filtered.head()

<ipython-input-12-56b5a7cdc849>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slight_partisan_filtered['slant'] = 1
<ipython-input-12-56b5a7cdc849>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slight_partisan_filtered['nonpartisan'] = 0
<ipython-input-12-56b5a7cdc849>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,title,slant,nonpartisan,slight-partisan,hyperpartisan
11,"2020 resolution for Trump, Democrats and us: L...",1,0,1,0
12,"New year, new laws, new (?) decade: 5 things y...",1,0,1,0
21,Protesters at US Embassy in Baghdad disperse a...,1,0,1,0
26,Chief Justice John Roberts: 'We have come to t...,1,0,1,0
27,Giuliani says he would be willing to testify i...,1,0,1,0


In [ ]:
df_slight_partisan_filtered.shape

(39210, 5)

In [ ]:
df_slight_partisan_filtered_rs = df_slight_partisan_filtered.sample(frac=0.20)

In [ ]:
df_slight_partisan_filtered_rs.shape

(7842, 5)

In [ ]:
df_partisan = pd.read_csv("Datasets/news_headlines_usa_biased.csv")

In [ ]:
df_partisan

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url
0,900954563,2020-04-15 00:00:00,One Day U: Going Back to College for One Day |...,https://www.huffingtonpost.com/renee-fisher/on...,en,False,NaN,27502,HuffPost,http://www.huffingtonpost.com/#
1,802518660,2020-11-15 00:00:00,Watch Donald Trump Compassionately Defend Drea...,https://www.huffingtonpost.com/entry/trump-def...,en,False,NaN,27502,HuffPost,http://www.huffingtonpost.com/#
2,869218430,2020-04-18 00:00:00,'Whitewashing War Crimes': How UK Academics Pr...,https://www.huffingtonpost.co.uk/entry/uk-acad...,en,False,NaN,27502,HuffPost,http://www.huffingtonpost.com/#
3,1001112150,2020-06-18 00:00:00,Mexican Official Says Girl With Down Syndrome ...,https://www.huffingtonpost.co.uk/entry/mexican...,en,False,NaN,27502,HuffPost,http://www.huffingtonpost.com/#
4,1483890044,NaN,"Todd Bensman, Author at The Federalist",https://thefederalist.com/author/toddbensman/,en,False,NaN,366282,Federalist,http://thefederalist.com/
...,...,...,...,...,...,...,...,...,...,...
45600,1863559557,2020-12-21 21:53:42,"Highlights of $900 billion COVID-19 relief, wr...",https://www.breitbart.com/news/highlights-of-9...,en,False,"education and schools, law and legislation, fi...",19334,Breitbart,http://www.breitbart.com
45601,1863924908,2020-10-27 22:34:41,"Nevada to Join California, Oregon, and Washing...",https://www.breitbart.com/health/2020/10/27/ne...,en,False,"politics and government, united states politic...",19334,Breitbart,http://www.breitbart.com
45602,1867816101,2020-12-13 03:59:10,Religious Freedom: Will U.S. Emphasis Continue...,https://amgreatness.com/2020/12/12/religious-f...,en,False,"politics and government, international relations",304983,American Greatness,http://amgreatness.com/
45603,1867897848,2020-12-11 10:27:57,U.S. COVID-19 Death Toll Now Higher Than Battl...,https://www.thedailybeast.com/us-covid-19-deat...,en,False,"medicine and health, diseases and conditions",1707,Daily Beast,http://www.thedailybeast.com/


In [ ]:
df_partisan['media_name'].value_counts()

Breitbart             11276
HuffPost               7467
Daily Caller           6687
Alternet               4885
Daily Beast            4611
Daily Wire             3170
Slate                  2502
American Greatness     1948
New Yorker             1617
Federalist             1442
Name: media_name, dtype: int64

In [ ]:
df_partisan_filtered = df_partisan.loc[df_partisan['media_name'].isin(['Breitbart', 'HuffPost', 'Daily Caller', 'Alternet', 'Daily Beast', 'Daily Wire'])]
df_partisan_filtered['media_name'].value_counts()

Breitbart       11276
HuffPost         7467
Daily Caller     6687
Alternet         4885
Daily Beast      4611
Daily Wire       3170
Name: media_name, dtype: int64

In [ ]:
df_partisan_filtered['slant'] = 2
df_partisan_filtered['nonpartisan'] = 0
df_partisan_filtered['slight-partisan'] = 0
df_partisan_filtered['hyperpartisan'] = 1
df_partisan_filtered = df_partisan_filtered.drop(['stories_id', 'publish_date', 'url', 'language', 'ap_syndicated', 'themes', 'media_id', 'media_name', 'media_url'], axis=1)
df_partisan_filtered.head()

<ipython-input-19-fa064ead8e4d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_partisan_filtered['slant'] = 2
<ipython-input-19-fa064ead8e4d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_partisan_filtered['nonpartisan'] = 0
<ipython-input-19-fa064ead8e4d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

,title,slant,nonpartisan,slight-partisan,hyperpartisan
0,One Day U: Going Back to College for One Day |...,2,0,0,1
1,Watch Donald Trump Compassionately Defend Drea...,2,0,0,1
2,'Whitewashing War Crimes': How UK Academics Pr...,2,0,0,1
3,Mexican Official Says Girl With Down Syndrome ...,2,0,0,1
5,Netflix’s Weird Show About a ‘Messiah’ From th...,2,0,0,1


In [ ]:
df_partisan_filtered.shape

(38096, 5)

In [ ]:
df_partisan_filtered_rs = df_partisan_filtered.sample(frac=0.20)

In [ ]:
df_partisan_filtered_rs.shape

(7619, 5)

In [ ]:
frames = [df_nonpartisan_filtered_rs, df_slight_partisan_filtered_rs, df_partisan_filtered_rs]
df = pd.concat(frames, ignore_index = True)
df

,title,slant,nonpartisan,slight-partisan,hyperpartisan
0,What you need to know about the coronavirus ri...,0,1,0,0
1,U.S. officials reiterate COVID-19 safety advic...,0,1,0,0
2,"A year into COVID-19, U.N. declares a day of '...",0,1,0,0
3,Bernie Sanders raises mammoth $46.5 million in...,0,1,0,0
4,World TeamTennis to allow up to 500 fans at ma...,0,1,0,0
...,...,...,...,...,...
22554,As Many As 35 Million People Haven’t Yet Recei...,2,0,0,1
22555,Buttigieg Campaign Removes Fundraiser Co-Host ...,2,0,0,1
22556,Exclusive--Tragesser: New York’s Green Light L...,2,0,0,1
22557,Ocasio-Cortez Defends ‘Defund The Police’ Slog...,2,0,0,1


In [ ]:
df['slant'].value_counts()

1    7842
2    7619
0    7098
Name: slant, dtype: int64

In [ ]:
df.rename(columns={'title': 'text'}, inplace=True)

In [ ]:
df.to_csv('Datasets/news_headlines_reduced_sample.csv', index = False)

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.30, random_state=42)
train_df.shape, test_df.shape

((15791, 5), (6768, 5))

In [ ]:
train_df.to_csv('Datasets/Splits/Train/news_headlines_usa_label_hyperpartisan_train.csv', index = False)

In [ ]:
test_df.to_csv('Datasets/Splits/Test/news_headlines_usa_label_hyperpartisan_test.csv', index = False)